<a href="https://colab.research.google.com/github/Rov-WIlde/SAN3-ERP-Code-Repository/blob/main/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install libraries for relatio model. Collab doesnt have a virutal environment, so each notebook is its own virtual environment. These libraries has been installed, so no need for downloading again WHILST working in this notebook

In [2]:
# Install specific versions - this is to make relatio work
!pip install huggingface_hub==0.10.1
!pip install transformers==4.24.0
!pip install accelerate==0.12.0
!pip install torch==1.12.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 73.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.20.1
    Uninstalling transformers-4.20.1:
      Successfully uninstalled transformers-4.20.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
allennlp 2.10.1 requires transformers<4.21,>=4.1, but you have transformers 4.24.0 which is incompatible.


  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.34.2
    Uninstalling accelerate-0.34.2:
      Successfully uninstalled accelerate-0.34.2


In [1]:
# install relevant packages
!pip install --upgrade pip wheel setuptools
!pip install -U relatio==0.3.0

  Using cached pip-24.2-py3-none-any.whl.metadata (3.6 kB)
ERROR: Operation cancelled by user
  Preparing metadata (setup.py) ... canceled
ERROR: Operation cancelled by user


In [ ]:
# version downloaded
# Successfully installed en-core-web-lg-3.7.1 --> MAYBE IGNORE THIS
# Successfully installed spacy-3.7.6 thinc-8.2.5
!pip install -U spacy==3.7.6
!python -m spacy download en_core_web_lg==3.7.1 # GOT BEST RESULTS WHEN THIS LINE DIDNT RUN - MAYBE DELETE IT.

  Using cached spacy-3.7.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (27 kB)
  Using cached thinc-8.2.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (15 kB)
Using cached spacy-3.7.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (29.0 MB)
Using cached thinc-8.2.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (922 kB)
  Attempting uninstall: thinc
    Found existing installation: thinc 8.0.17
    Uninstalling thinc-8.0.17:
      Successfully uninstalled thinc-8.0.17
  Attempting uninstall: spacy
    Found existing installation: spacy 3.3.3
    Uninstalling spacy-3.3.3:
      Successfully uninstalled spacy-3.3.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
allennlp 2.10.1 requires spacy<3.4,>=2.1.0, but you have spacy 3.7.6 which is incompatible.

✘ No compatible package found for 'en_co

In [ ]:
# version downloaded
# Successfully installed BERTopic-0.16.3
!pip install BERTopic==0.16.3

In [2]:
# download this - will impact results of BERTopic so need it.
!pip install numpy==1.23.5


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 100.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
accelerate 0.34.2 requires huggingface-hub>=0.21.0, but you have huggingface-hub 0.10.1 which is incompatible.
albucore 0.0.16 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
albumentations 1.4.15 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
albumentations 1.4.15 requires pydantic>=2.7.0, but you have pydantic 1.8.2 which is incompatible.
bigframes 1.17.0 requires numpy>=1.24.0, but you have numpy 1.23.5 which is incompatible.
chex 0.1.86 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.
pandas-stubs 2.1.4.231227 r

In [1]:
import pandas as pd, numpy as np
import re
import torch
torch.cuda.is_available()
from google.colab import drive
drive.mount('/content/drive')
import spacy
from relatio import Preprocessor


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Importating and Processing Sentencizer segmented manifestos




In [2]:

# List of file paths
file_paths = [
        "/content/drive/MyDrive/clean_manifestos/con_01_clean.txt",
        "/content/drive/MyDrive/clean_manifestos/con_05_clean.txt",
        "/content/drive/MyDrive/clean_manifestos/con_10_clean.txt",
        "/content/drive/MyDrive/clean_manifestos/con_24_clean(new).txt",
        "/content/drive/MyDrive/clean_manifestos/lab_05_clean.txt",
        "/content/drive/MyDrive/clean_manifestos/lab_10_clean(new).txt",
        "/content/drive/MyDrive/clean_manifestos/lab_19_clean.txt",
        "/content/drive/MyDrive/clean_manifestos/lab_24_clean(new).txt"
]

# Function to read files and carry out basic text-pre-processing
def read_file_without_newlines(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    # Use re.sub for regex-based replacements - removing page numbers
    content = re.sub(r'\n\d{1,3}\n', '', content) # remove 1-3 digits surrounded by newlines (page nums)
    content = re.sub(r'\d{1,3}  ', '', content) # remove 1-3 digits whn followed by a double space (page nums)
    # Remove patterns like "4:2" (lab 2010 manifesto page number format)
    content = re.sub(r'\d+:\d+', '', content)
    # replace newline and tab characters with spaces
    content = content.replace('\n', ' ')
    content = content.replace('\t', ' ')
    # remove different versions of bullet points (bullet points statements usually have '.' so can still be segmented)
    content = content.replace('❱', '')
    content = content.replace('•', '')
    return content

# Function to remove excessive spaces
def clean_up_text(text):
    # Remove spaces after punctuation that were newlines before (handles excessive spacing after deleting newlines)
    text = re.sub(r'([.,;?!]) +', r'\1 ', text)
    return text

# List to store cleaned texts
cleaned_texts = []

# Process each file using custom functions
for file_path in file_paths:
    text = read_file_without_newlines(file_path)
    cleaned_text = clean_up_text(text)
    cleaned_texts.append(cleaned_text)

# Create DataFrame
df = pd.DataFrame(cleaned_texts, columns=['manifesto'])

# Display the DataFrame
print(df)


                                           manifesto
0  time for common sense - the Conservative manif...
1  ARE YOU THINKING WHAT WE'RE THINKING? IT'S TIM...
2  INVITATION TO JOIN THE GOVERNMENT OF BRITAIN  ...
3  The Conservative and Unionist Party  We are re...
4  Britain forward not back. Preface by Tony Blai...
5   A future fair for all  Foreword from Gordon B...
6  FOREWORD This election will shape our country ...
7  Change   My plan for change This election is a...


Using Relatio package (Ash et al 2023) Preprocessor function in order to sentence segment manifestos

In [3]:
# Using Relatio package (Ash et al 2023) Preprocessor function in order to sentence segment manifestos

# Relatio package requires a 'doc' and 'id' column to work, so alter datafram to add this.
df.rename(columns={'manifesto': 'doc'}, inplace=True)
df['id'] = df.index
df.reset_index(drop=True, inplace=True)
df= df[['id', 'doc']]
# convert dataframe into a format useable by Relatio package

p = Preprocessor(
    spacy_model = "en_core_web_lg",
    remove_punctuation = False,
    remove_digits = False,
    lowercase = False,
    lemmatize = False,
    n_process = -1
)
sentence_df = p.split_into_sentences(
    df, output_path = None, progress_bar = True
)
sentence_df

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
Splitting into sentences...


  0%|          | 0/8 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|██████████| 8/8 [00:12<00:00,  1.60s/it]


,id,sentence
0,0,time for common sense - the Conservative manif...
1,0,"Its aim is to release the wisdom, decency and ..."
2,0,We can achieve that by handing back to individ...
3,0,We will free entrepreneurs to build businesses...
4,0,We want to set people free so that they have g...
...,...,...
8249,7,And with a renewed confidence that the future ...
8250,7,This manifesto contains our plan to begin that...
8251,7,To succeed we will need every individual and c...
8252,7,"We must come together, after 14 years of chaos..."


In [4]:
# Adding party and year ID column to Sentencizer sentence-segmented manifestos
# Define the mapping dictionary
id_mapping = {
    0: 'con_01',
    1: 'con_05',
    2: 'con_10',
    3: 'con_24',
    4: 'lab_05',
    5: 'lab_10',
    6: 'lab_19',
    7: 'lab_24'
}

# Apply the mapping to the 'id' column
sentence_df['id'] = sentence_df['id'].replace(id_mapping)

# Display the mod
sentence_df

,id,sentence
0,con_01,time for common sense - the Conservative manif...
1,con_01,"Its aim is to release the wisdom, decency and ..."
2,con_01,We can achieve that by handing back to individ...
3,con_01,We will free entrepreneurs to build businesses...
4,con_01,We want to set people free so that they have g...
...,...,...
8249,lab_24,And with a renewed confidence that the future ...
8250,lab_24,This manifesto contains our plan to begin that...
8251,lab_24,To succeed we will need every individual and c...
8252,lab_24,"We must come together, after 14 years of chaos..."


# Importing and processing MRP sentence-segmented manifestos
- this code returns each csv file as a dataframe along with its shape to check how many sentences it gives
- each sentence will have a id value to track the manifesto it came from e.g. 'lab_17'

In [5]:
# Integrating manifestos that have been manually segmented into sentences by Manifesto Research Project's annotators

# List of file paths
file_paths = [
    '/content/drive/MyDrive/clean_manifestos/con_15_sents.csv',
    '/content/drive/MyDrive/clean_manifestos/con_17_sents.csv',
    '/content/drive/MyDrive/clean_manifestos/con_19_sents.csv',
    '/content/drive/MyDrive/clean_manifestos/lab_01_sents.csv',
    '/content/drive/MyDrive/clean_manifestos/lab_15_sents.csv',
    '/content/drive/MyDrive/clean_manifestos/lab_17_sents.csv',
]

# Initialize a dictionary to hold dataframes
dataframes = {}

# Function to read a CSV file with different encodings and handle errors
def read_csv_with_handling(file_path, encodings=['utf-8', 'latin1', 'ISO-8859-1', 'cp1252']):
    for encoding in encodings:
        try:
            df = pd.read_csv(file_path, encoding=encoding, on_bad_lines='skip')
            return df
        except (UnicodeDecodeError, pd.errors.ParserError):
            continue
    raise ValueError(f"Unable to read {file_path} with provided encodings")

# Loop through file paths
for file_path in file_paths:
    # Read each CSV file with different encodings and error handling
    df = read_csv_with_handling(file_path)
    # Extract the file name without extension and use it as the key in the dictionary
    file_key = file_path.split('/')[-1].replace('_sents.csv', '')
    # Add the dataframe to the dictionary
    dataframes[file_key] = df


# CODE TO CHECK THE OUTPUT:
# Display the shape and head of each dataframe for examination
for key, df in dataframes.items():
    print(f"DataFrame for {key}:")
    print(f"Shape: {df.shape}")
    print(df.head(), "\n")

DataFrame for con_15:
Shape: (1867, 4)
                                                text  text_en  cmp_code  \
0              THE CONSERVATIVE PARTY MANIFESTO 2015      NaN       NaN   
1  ContentsFOREWORD by David Cameron – Leader of ...      NaN       NaN   
2       We have a plan for every stage of your life:      NaN       NaN   
3  FOR THE BEST START IN LIFE we will continue to...      NaN     504.0   
4  provide 7-day a week access to your GP and del...      NaN     504.0   

   eu_code  
0      NaN  
1      NaN  
2      NaN  
3      NaN  
4      NaN   

DataFrame for con_17:
Shape: (1612, 4)
                                                text  text_en cmp_code  \
0                              FIVE GIANT CHALLENGES      NaN        H   
1                       Strong and stable leadership      NaN        H   
2  Like generations before us, we are living thro...      NaN    305.3   
3  At such moments, our country requires strong a...      NaN    305.3   
4  Since the referend

combining all the sentences together into a single df FROM THE CSV FILES ONLY

In [6]:
# COMBINING ALL MANUALLY SEGMENTED MANIFESTOS INTO ONE DATAFRAME

# Convert each manually sentence-segmented manifesto into its own dataframe
con_15_df = dataframes['con_15']
con_17_df = dataframes['con_17']
con_19_df = dataframes['con_19']
lab_01_df = dataframes['lab_01']
lab_15_df = dataframes['lab_15']
lab_17_df = dataframes['lab_17']

# ADDING 'id' column and renaming 'text' to 'sentence' so all sentences can be tracked to original manifesto
def prepare_dataframe(df, id_prefix):
    df['id'] = id_prefix
    df.rename(columns={'text': 'sentence'}, inplace=True)
    return df[['id', 'sentence']]

con_15_df = prepare_dataframe(con_15_df, 'con_15')
con_17_df = prepare_dataframe(con_17_df, 'con_17')
con_19_df = prepare_dataframe(con_19_df, 'con_19')
lab_01_df = prepare_dataframe(lab_01_df, 'lab_01')
lab_15_df = prepare_dataframe(lab_15_df, 'lab_15')
lab_17_df = prepare_dataframe(lab_17_df, 'lab_17')

# COMBINING ALL MANUALLY-SEGMENTED MANIFESTOS INTO A SINGLE DATAFRAME
combined_df = pd.concat([con_15_df, con_17_df, con_19_df, lab_01_df, lab_15_df, lab_17_df], ignore_index=True)

# Display the combined dataframe
print(f"Combined DataFrame shape: {combined_df.shape}")
combined_df


Combined DataFrame shape: (9027, 2)


,id,sentence
0,con_15,THE CONSERVATIVE PARTY MANIFESTO 2015
1,con_15,ContentsFOREWORD by David Cameron – Leader of ...
2,con_15,We have a plan for every stage of your life:
3,con_15,FOR THE BEST START IN LIFE we will continue to...
4,con_15,provide 7-day a week access to your GP and del...
...,...,...
9022,lab_17,We will extend the remit of the Groceries Code...
9023,lab_17,At least a billion people around the world can...
9024,lab_17,We will establish a Centre for Universal Healt...
9025,lab_17,We will invest in new public-health driven res...


# MERGE ALL MANIFESTOS into a single dataframe. This is the working corpus for TM

In [7]:
# Combine all dataframes into a single dataframe
sentence_df = pd.concat([sentence_df,combined_df ], ignore_index=True)
sentence_df

,id,sentence
0,con_01,time for common sense - the Conservative manif...
1,con_01,"Its aim is to release the wisdom, decency and ..."
2,con_01,We can achieve that by handing back to individ...
3,con_01,We will free entrepreneurs to build businesses...
4,con_01,We want to set people free so that they have g...
...,...,...
17276,lab_17,We will extend the remit of the Groceries Code...
17277,lab_17,At least a billion people around the world can...
17278,lab_17,We will establish a Centre for Universal Healt...
17279,lab_17,We will invest in new public-health driven res...


Save the Dataframe containing the working corpus

In [8]:
# Specify the folder path where the CSV file will be saved
folder_path = "/content/drive/MyDrive/clean_manifestos/"

# Specify the file name you want to save the DataFrame as
file_name = "sentence_df.csv"

# Save the DataFrame as a CSV file in the specified folder
sentence_df.to_csv(folder_path + file_name, index=False)
